In [ ]:
import io
import pandas as pd
import polars as pl
import msoffcrypto
from openpyxl import load_workbook

# --- CONFIGURACIÓ ---
ruta_excel = r'ruta/del/arxiu.xlsx'
tu_contraseña = '*****'

# --- Desxifrar arxiu ---
decrypted_workbook = io.BytesIO()
with open(ruta_excel, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=tu_contraseña)
    office_file.decrypt(decrypted_workbook)

# --- Obtindre noms dels fulls ---
decrypted_workbook.seek(0)
wb = load_workbook(decrypted_workbook, read_only=True)
nombres_hojas = wb.sheetnames
print("▶ Fulls trobats:")
for i, nombre in enumerate(nombres_hojas):
    print(f"{i + 1}: {nombre}")

# --- Llegir només el full núm 5 ---
indice_hoja = 5  # Full 5 
nombre_hoja5 = nombres_hojas[indice_hoja]

decrypted_workbook.seek(0)
df_hoja5 = pd.read_excel(decrypted_workbook, sheet_name=nombre_hoja5, engine='openpyxl')
df = df_hoja5

# --- Mostrar primeres files del full 5 ---
print(f"\n Capçalera del full 5: {nombre_hoja5}")
print(df_hoja5.head())

# PAS 1: Preparar el DataFrame
if isinstance(df, pl.DataFrame):
    df_pl = df
else:
    df_pl = pl.from_pandas(df)

print(f" Dataset original: {df_pl.shape}")

# PAS 2: Definir estratègia de fusió per cada variable
def crear_columna_fusionada(df, columnas_candidatas, nombre_resultado):
    """Crea una columna fusionada prenent el primer valor no-null de les candidates"""
    columnas_existentes = [col for col in columnas_candidatas if col in df.columns]
    
    if not columnas_existentes:
        return df.with_columns(pl.lit(None).alias(nombre_resultado))
    
    # Usar coalesce per prendre el primer valor no-null
    expresion = pl.coalesce([pl.col(col) for col in columnas_existentes])
    return df.with_columns(expresion.alias(nombre_resultado))

# PAS 3: Fusionar totes les variables duplicades
print("\n FUSIONANT VARIABLES DUPLICADES:")

# Fusionar Albúmina
df_fusionado = crear_columna_fusionada(
    df_pl, 
    ['ALBÚMINA', 'resultadoALBÚMINA'], 
    'ALBUMINA_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['fechaResultado_ALBUMINA', 'fechaResultadoALBÚMINA'],
    'FECHA_ALBUMINA_FUSIONADA'
)

# Fusionar PCR
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['PCR', 'resultadoPCR'],
    'PCR_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['fechaResultadoPCR', 'fechaResultadoPCR1'],
    'FECHA_PCR_FUSIONADA'
)

# Fusionar Creatinina
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['resultadoCREATININA', 'resultadoCREATININA1'],
    'CREATININA_FUSIONADA'
)

# Per a creatinina només hi ha una data
df_fusionado = df_fusionado.with_columns(
    pl.col('fechaResultadoCREATININA').alias('FECHA_CREATININA_FUSIONADA')
)

# Fusionar dades demogràfiques duplicades
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descTipoPaciente', 'descTipoPaciente1'],
    'TIPO_PACIENTE_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descLinea', 'descLinea1'],
    'LINEA_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descEsquema', 'descEsquema1'],
    'ESQUEMA_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['codigoPostal', 'codigoPostal1'],
    'CODIGO_POSTAL_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descHospitalCrc', 'descHospitalCrc1'],
    'HOSPITAL_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descZona', 'descZona1'],
    'ZONA_FUSIONADA'
)

# PAS 4: Verificar quantes dades rescatem
print(" DADES RESCATADES:")
print(f"Albúmina fusionada: {df_fusionado['ALBUMINA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"PCR fusionada: {df_fusionado['PCR_FUSIONADA'].drop_nulls().len():,} valors") 
print(f"Creatinina fusionada: {df_fusionado['CREATININA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Data Albúmina fusionada: {df_fusionado['FECHA_ALBUMINA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Data PCR fusionada: {df_fusionado['FECHA_PCR_FUSIONADA'].drop_nulls().len():,} valors")

# PAS 5: Crear el mapatge per a la transformació
variables_fusionadas = {
    'Rockwood': {'valor': 'Rockwood', 'fecha': 'F_Rockwood'},
    'ALBUMINA': {'valor': 'ALBUMINA_FUSIONADA', 'fecha': 'FECHA_ALBUMINA_FUSIONADA'},
    'PCR': {'valor': 'PCR_FUSIONADA', 'fecha': 'FECHA_PCR_FUSIONADA'},
    'CREATININA': {'valor': 'CREATININA_FUSIONADA', 'fecha': 'FECHA_CREATININA_FUSIONADA'},
    'EUROQOL': {'valor': 'descResultadoEscala1', 'fecha': 'fechaEUROQOL'}
}

# Columnes demogràfiques fusionades
cols_demograficas = ['SIPCOD', 'sexo', 'fecha Naci', 'fecha Exitus',
                     'TIPO_PACIENTE_FUSIONADO', 'LINEA_FUSIONADA', 'ESQUEMA_FUSIONADO',
                     'CODIGO_POSTAL_FUSIONADO', 'HOSPITAL_FUSIONADO', 'ZONA_FUSIONADA']

print(f"\n LLESTOS PER TRANSFORMAR AMB TOTES LES VARIANTS FUSIONADES")
print(f"Variables a processar: {list(variables_fusionadas.keys())}")

# PAS 6: Transformació a format llarg amb dades fusionades
print("\n TRANSFORMANT A FORMAT LLARG:")

dfs_lista = []

# Processar variables simples (no EuroQol)
for variable, config in variables_fusionadas.items():
    if variable != 'EUROQOL':
        col_valor = config['valor']
        col_fecha = config['fecha']
        
        print(f"Processant {variable}...")
        
        # Filtrar només registres amb dades vàlides
        df_temp = df_fusionado.filter(
            (pl.col(col_valor).is_not_null()) & 
            (pl.col(col_fecha).is_not_null())
        ).select([
            pl.col('SIPCOD').cast(pl.Utf8),
            pl.col(col_fecha).cast(pl.Utf8).alias('Fecha'),
            pl.col(col_valor).cast(pl.Utf8).alias('Valor'),
            *[pl.col(col).cast(pl.Utf8) for col in cols_demograficas[1:]]
        ]).with_columns(
            pl.lit(variable).alias('Variable')
        )
        
        dfs_lista.append(df_temp)
        print(f"   {variable}: {len(df_temp):,} registres vàlids")

# Unir variables simples
if dfs_lista:
    df_simple_fusionado = pl.concat(dfs_lista)
    print(f"\n Variables simples unides: {len(df_simple_fusionado):,} files")
else:
    df_simple_fusionado = pl.DataFrame()

# PAS 7: Processar EuroQol
print("\nProcessant EuroQol...")
euroqol_config = variables_fusionadas['EUROQOL']

euroqol_df = df_fusionado.filter(
    (pl.col(euroqol_config['valor']).is_not_null()) & 
    (pl.col(euroqol_config['fecha']).is_not_null())
).select([
    pl.col('SIPCOD').cast(pl.Utf8),
    pl.col(euroqol_config['fecha']).cast(pl.Utf8).alias('Fecha'),
    pl.col(euroqol_config['valor']).cast(pl.Utf8).alias('Valor_Original'),
    *[pl.col(col).cast(pl.Utf8) for col in cols_demograficas[1:]]
])

print(f"Dataset EuroQol: {len(euroqol_df):,} files")

# Expandir EuroQol 
if len(euroqol_df) > 0:
    euroqol_expandido = euroqol_df.with_columns([
        pl.col('Valor_Original').str.strip_chars().str.split('/').list.first().alias('valor_limpio')
    ]).filter(
        pl.col('valor_limpio').is_not_null() & 
        (pl.col('valor_limpio').str.len_chars() == 5)
    ).with_columns([
        pl.col('valor_limpio').str.slice(0, 1).alias('EQ5D_Movilidad'),
        pl.col('valor_limpio').str.slice(1, 1).alias('EQ5D_CuidadoPersonal'), 
        pl.col('valor_limpio').str.slice(2, 1).alias('EQ5D_ActividadesHabituales'),
        pl.col('valor_limpio').str.slice(3, 1).alias('EQ5D_DolorMalestar'),
        pl.col('valor_limpio').str.slice(4, 1).alias('EQ5D_AnsiedadDepresion')
    ])
    
    dimensiones_eq5d = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
                        'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']
    
    
    euroqol_largo = euroqol_expandido.unpivot(
        index=['SIPCOD', 'Fecha'] + cols_demograficas[1:],
        on=dimensiones_eq5d,
        variable_name='Variable',
        value_name='Valor'
    )
    print(f"   EuroQol expandit: {len(euroqol_largo):,} files")
else:
    euroqol_largo = pl.DataFrame()

# PAS 8: Unir tot
dataframes_finales = []
if len(df_simple_fusionado) > 0:
    dataframes_finales.append(df_simple_fusionado)
    print(f"\n Variables simples: {len(df_simple_fusionado):,} files")

if len(euroqol_largo) > 0:
    dataframes_finales.append(euroqol_largo)  
    print(f" EuroQol: {len(euroqol_largo):,} files")

if dataframes_finales:
    # Assegurar columnes consistents
    columnas_comunes = None
    for df in dataframes_finales:
        if columnas_comunes is None:
            columnas_comunes = df.columns
        else:
            columnas_comunes = [col for col in columnas_comunes if col in df.columns]
    
    print(f"Columnes comunes: {len(columnas_comunes)}")
    
    dataframes_reordenados = [df.select(columnas_comunes) for df in dataframes_finales]
    df_final_mejorado = pl.concat(dataframes_reordenados)
    
    print(f"\n RESULTAT MILLORAT: {len(df_final_mejorado):,} files totals")
    print(f" Pacients únics: {df_final_mejorado['SIPCOD'].n_unique():,}")
    
    # Ordenar
    df_final_mejorado = df_final_mejorado.sort(['SIPCOD', 'Fecha'])
    
    # Omplir dades demogràfiques
    cols_rellenar = [col for col in cols_demograficas[1:] if col in df_final_mejorado.columns]
    if cols_rellenar:
        df_final_mejorado = df_final_mejorado.with_columns([
            pl.col(col).fill_null(strategy="forward").fill_null(strategy="backward").over('SIPCOD')
            for col in cols_rellenar
        ])
    
    # Eliminar duplicats
    df_final_mejorado = df_final_mejorado.unique()
    
    print(f" RESULTAT FINAL: {df_final_mejorado.shape}")
    
    # Mostrar distribució per variable
    print(f"\n DISTRIBUCIÓ FINAL PER VARIABLE:")
    conteo_final = df_final_mejorado.group_by('Variable').agg(pl.len().alias('count')).sort('count', descending=True)
    print(conteo_final)
    
    print(f"\n MILLORA ACONSEGUIDA:")
    print(f"Abans: ~33,805 files")
    print(f"Ara: {len(df_final_mejorado):,} files")
    if len(df_final_mejorado) > 0:
        mejora = len(df_final_mejorado) - 33805
        porcentaje = ((len(df_final_mejorado) / 33805) - 1) * 100
        print(f" Increment: +{mejora:,} files ({porcentaje:.1f}% més dades!)")
    
else:
    print(" No s'han pogut processar les dades")
    df_final_mejorado = pl.DataFrame()

print("\n Procés completat amb fusió intel·ligent de dades!")

# Assignar el resultat final al DataFrame principal
df = df_final_mejorado

# Classificació correcta de variables
variables_euroqol = df.filter(pl.col("Variable").str.starts_with("EQ5D"))["Variable"].unique()
variables_numericas = ["ALBUMINA", "PCR", "CREATININA", "Rockwood"]
variables_ordinales = variables_euroqol.to_list()

# Neteja amb càlculs temporals clau
df_clean = df.with_columns([
    pl.col("Fecha").str.to_datetime(),
    pl.col("fecha Naci").str.to_datetime(),
    pl.col("fecha Exitus").str.to_datetime(),
    
    # Valors segons tipus de variable
    pl.when(pl.col("Variable").is_in(variables_numericas))
      .then(pl.col("Valor").cast(pl.Float64, strict=False))
      .otherwise(None).alias("valor_numerico"),
    
    pl.when(pl.col("Variable").is_in(variables_ordinales))
      .then(pl.col("Valor").cast(pl.Int32, strict=False))
      .otherwise(None).alias("valor_ordinal")
]).with_columns([
    # Edat al moment de la mesura
    ((pl.col("Fecha") - pl.col("fecha Naci")).dt.total_days()).alias("edad_dias_medicion"),
    
    # Estat vital
    pl.col("fecha Exitus").is_not_null().alias("fallecido"),
    
    # Temps fins a defunció o fins a última mesura (si viu)
    pl.when(pl.col("fecha Exitus").is_not_null())
      .then((pl.col("fecha Exitus") - pl.col("Fecha")).dt.total_days())
      .otherwise(None).alias("dias_hasta_exitus"),
    
    # Temps de supervivència des de la mesura fins a esdeveniment o censura
    pl.when(pl.col("fecha Exitus").is_not_null())
      .then((pl.col("fecha Exitus") - pl.col("Fecha")).dt.total_days())
      .otherwise((pl.lit("2025-06-27").str.to_date() - pl.col("Fecha").dt.date()).dt.total_days())
      .alias("tiempo_supervivencia")
])

print("\n DATASET FINAL NET I PREPARAT PER A ANÀLISI")
print(f"Forma final: {df_clean.shape}")
print(f"Columnes disponibles: {df_clean.columns}")
print(f"Variables numèriques: {variables_numericas}")
print(f"Variables ordinals EuroQol: {variables_ordinales}")

# Llegendes 
# Rosa = Morts, Blau = Vius
colors_llegenda = {
    'fallecido': {
        True: {'color': 'hotpink', 'label': 'Morts'},
        False: {'color': 'dodgerblue', 'label': 'Vius'}
    },
    'estat_vital': {
        'Mort': {'color': 'hotpink', 'label': 'Morts'},
        'Viu': {'color': 'dodgerblue', 'label': 'Vius'}
    }
}

In [ ]:
# Llistat de tipus de paciente a conservar
tipos_a_mantener = [
    "Crónico pluripatológico",
"Paliativo no oncológico",
"Paliativo oncológico"
]

# Aplicar el filtro en polars
df_filtrado_polars = df_clean.filter(
    pl.col("TIPO_PACIENTE_FUSIONADO").is_in(tipos_a_mantener)
)


In [ ]:
df_clean=df_filtrado_polars

In [ ]:
# 4.1 - ANÀLISI DESCRIPTIU 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

# Llistat de tipus de paciente a conservar
tipos_a_mantener = [
    "Crónico pluripatológico",
"Paliativo no oncológico",
"Paliativo oncológico"
]

# Configuració d'estils globals
sns.set_style("white")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# Configuració del període d'estudi
PERIODE_INICI = "2020"
PERIODE_FI = "2024"
PERIODE_COMPLET = f"{PERIODE_INICI}-{PERIODE_FI}"

# Esquema de colors
colors_variables = {
    'rockwood_mitjana': '#FF9999',      # Rosa 
    'eq_index_esp': '#E6B3FF',          # Lila 
    'ALBUMINA': '#808080',              # Gris
    'PCR': '#808080',                   # Gris
    'CREATININA': '#808080'             # Gris
}


#  RECOMPTE I DISTRIBUCIÓ DE VARIABLES
# ===================================================================

# 1) Convertir df_clean a pandas 
if 'df_pd' not in locals():
    if hasattr(df_clean, 'to_pandas'):
        df_pd = df_clean.to_pandas()
    else:
        df_pd = df_clean.copy()
    df_pd["edat_anys"] = df_pd["edad_dias_medicion"] / 365.25

# Variables EuroQol
eq5d_vars = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
             'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']

# Filtrar només EuroQol
df_euroqol = df_pd[df_pd['Variable'].isin(eq5d_vars)]

print(f"Registres EuroQol totals: {len(df_euroqol):,}")
print(f"Pacients amb EuroQol: {df_euroqol['SIPCOD'].nunique():,}")

# Filtrar només mesures completes (5 dimensions per mesura)
mesures_completes = (df_euroqol.groupby(['SIPCOD', 'Fecha'])['Variable']
                     .nunique().reset_index(name='n_dimensions'))
mesures_5d = mesures_completes[mesures_completes['n_dimensions'] == 5]

if len(mesures_5d) > 0:
    mesures_5d['key'] = mesures_5d['SIPCOD'].astype(str) + '_' + mesures_5d['Fecha'].astype(str)
    df_euroqol['key'] = df_euroqol['SIPCOD'].astype(str) + '_' + df_euroqol['Fecha'].astype(str)
    df_euroqol_clean = df_euroqol[df_euroqol['key'].isin(mesures_5d['key'])].copy()
    
    print(f"Mesures completes (5 dimensions): {len(mesures_5d):,}")
    print(f"Registres després de neteja: {len(df_euroqol_clean):,}")
else:
    df_euroqol_clean = df_euroqol.copy()
    print("No s'han trobat mesures completes, usant totes les dades disponibles")

# Calcular nombre total de pacients per consistència
print(df_pd.columns)
print(df_pd.head())
df_pd.columns = ['SIPCOD', 'Fecha', 'Valor', 'sexo', 'fecha Naci', 'fecha Exitus',
       'TIPO_PACIENTE_FUSIONADO', 'LINEA_FUSIONADA', 'ESQUEMA_FUSIONADO',
       'CODIGO_POSTAL_FUSIONADO', 'HOSPITAL_FUSIONADO', 'ZONA_FUSIONADA',
       'Variable', 'valor_numerico', 'valor_ordinal', 'edad_dias_medicion',
       'fallecido', 'dias_hasta_exitus', 'tiempo_supervivencia', 'edat_anys']
TOTAL_PACIENTS_CLINIC_1 = df_pd['SIPCOD'].dropna()
TOTAL_PACIENTS_CLINIC_2 = TOTAL_PACIENTS_CLINIC_1.nunique()
print(f" DATASET CLÍNIC: {TOTAL_PACIENTS_CLINIC_2:,} pacients | Període: {PERIODE_COMPLET}")

# 2) FILTRAR OBSERVACIONS EQ5D INCONSISTENTS (SIMPLIFICAT)
print("Filtrant mesures EQ5D incompletes...")

# Identificar mesures EQ5D completes per pacient+data
eq5d_data = df_pd[df_pd["Variable"].str.startswith("EQ5D")].copy()

if len(df_euroqol_clean) > 0:
    
    # Filtrar EQ5D consistent
    df_euroqol['key'] = df_euroqol['SIPCOD'].astype(str) + '_' + df_euroqol['Fecha'].astype(str)
    eq5d_consistent = df_euroqol[df_euroqol['key'].isin(mesures_5d['key'])]
    
    # Combinar amb altres variables
    df_altres = df_pd[~df_pd['Variable'].isin(eq5d_vars)]
    df_net = pd.concat([df_altres, df_euroqol_clean], ignore_index=True)
    
    print(f"   Mesures EQ5D eliminades (incompletes): {len(df_euroqol) - len(df_euroqol_clean):,}")
    print(f"   Mesures EQ5D completes mantingudes: {len(df_euroqol_clean):,}")
else:
    df_net = df_pd.copy()
    eq5d_consistent = pd.DataFrame()
    print("No hi ha dades EQ5D per filtrar")

# 3) Comptar mesures per variable
counts = df_net['Variable'].value_counts().sort_values(ascending=False).reset_index()
counts.columns = ['Variable', 'n_mesures']

print(f"   Total observacions després del filtratge: {len(df_net):,}")

# 4) Gràfic: Nombre de mesures per variable 
plt.figure(figsize=(12, max(8, len(counts) * 0.4)))

# Assignar colors
colors = []
for var in counts['Variable']:
    if var.startswith('EQ5D'):
        colors.append('#E6B3FF')  # Lila 
    elif var == 'Rockwood':
        colors.append('#FF9999')  # Rosa 
    elif var in ['ALBUMINA', 'PCR', 'CREATININA']:
        colors.append('#808080')  # Gris 
    else:
        colors.append('#66B3FF')  # Blau 

# Gràfic 
bars = plt.barh(range(len(counts)), counts['n_mesures'], color=colors, edgecolor='black', linewidth=0.5)

plt.xlabel("Nombre de mesures", fontsize=12)
plt.ylabel("Tipus de mesura", fontsize=12)
plt.title(f"Nombre total de mesures per tipus de variable\n"
          f"Mesures totals: {len(df_net):,} | Període: {PERIODE_COMPLET}", 
          fontweight='bold', fontsize=14)

# Etiquetes de variables
plt.yticks(range(len(counts)), counts['Variable'])

# Afegir valors a les barres
for i, (bar, val) in enumerate(zip(bars, counts['n_mesures'])):
    plt.text(bar.get_width() + counts['n_mesures'].max() * 0.01, 
             bar.get_y() + bar.get_height()/2, 
             f"{val:,}", va='center', fontsize=10, fontweight='bold')

plt.xlim(0, counts['n_mesures'].max() * 1.15)
plt.tight_layout()
plt.show()

pacients_counts = df_net.groupby('Variable')['SIPCOD'].nunique().sort_values(ascending=False).reset_index()
pacients_counts.columns = ['Variable', 'n_pacients']
total_pacients= df_net['SIPCOD'].nunique()


# Assignar colors segons variable 
colors_pac = []
for var in pacients_counts['Variable']:
    if var.startswith('EQ5D'):
        colors_pac.append('#E6B3FF')  # Lila 
    elif var == 'Rockwood':
        colors_pac.append('#FF9999')  # Rosa 
    elif var in ['ALBUMINA', 'PCR', 'CREATININA']:
        colors_pac.append('#808080')  # Gris
    else:
        colors_pac.append('#66B3FF')  # Blau


# CARACTERÍSTIQUES CLÍNIQUES PER PACIENTS ÚNICS
# ===================================================================

# Càlcul de l'índex EuroQol 
def calcular_euroqol_espanyol(row):
    """Calcula l'índex EuroQol-5D segons els coeficients espanyols"""
    cols = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
            'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']
    
    # Verificar que totes les columnes existeixen i no són nul·les
    if not all(col in row.index and pd.notna(row[col]) for col in cols):
        return np.nan
    
    try:
        valores = [int(row[col]) for col in cols]
        m, c, a, d, an = valores
        
        # Si tots són 1, índex perfecte
        if all(x == 1 for x in valores):
            return 1.0
        
        # Coeficients espanyols (EQ-5D-3L)
        decrements = {
            'movilidad': [0, 0.0897, 0.1794],      
            'cuidado': [0, 0.1012, 0.2024],
            'actividades': [0, 0.0551, 0.1102], 
            'dolor': [0, 0.0596, 0.1192],
            'ansiedad': [0, 0.0512, 0.1024]
        }
        
        # Calcular decrement total
        decrement_total = (decrements['movilidad'][min(m-1, 2)] + 
                          decrements['cuidado'][min(c-1, 2)] + 
                          decrements['actividades'][min(a-1, 2)] + 
                          decrements['dolor'][min(d-1, 2)] + 
                          decrements['ansiedad'][min(an-1, 2)])
        
        # Penalització addicional si alguna dimensió és nivell 3
        if any(x == 3 for x in valores):
            decrement_total += 0.2119
        
        # Aplicar fórmula: 1 - constant - decrements
        index_eq = 1 - 0.1502 - decrement_total
        
        # L'índex negatiu (estats pitjors que la mort)
        return max(index_eq, -0.5)  
    except:
        return np.nan

# Agregar dades per pacient
print("Agregant dades per pacient...")

# Variables bàsiques per pacient (primera/última/mitjana segons el cas)
agg_pacient = df_pd.groupby("SIPCOD").agg({
    'Variable': 'count',  # nombre de mesures
    'TIPO_PACIENTE_FUSIONADO': 'first',
    'fallecido': 'first',
    'edat_anys': 'mean'
}).rename(columns={
    'Variable': 'n_mesures',
    'TIPO_PACIENTE_FUSIONADO': 'patologia',
    'edat_anys': 'edat_mitjana'
})

# Rockwood mitjana
rockwood_data = df_pd[df_pd["Variable"] == "Rockwood"]
if len(rockwood_data) > 0:
    rock_mitjana = rockwood_data.groupby("SIPCOD")["valor_numerico"].mean()
    agg_pacient = agg_pacient.join(rock_mitjana.rename('rockwood_mitjana'), how='left')
else:
    agg_pacient['rockwood_mitjana'] = np.nan

# EuroQol amb fórmula 
if len(eq5d_consistent) > 0:
    # Pivot EQ5D per aplicar la fórmula
    eq_pivot = (eq5d_consistent.pivot_table(
        index=["SIPCOD", "Fecha"], 
        columns="Variable", 
        values="valor_ordinal", 
        aggfunc="first"
    ).reset_index())
    
    # Aplicar fórmula 
    eq_pivot["eq_index_esp"] = eq_pivot.apply(calcular_euroqol_espanyol, axis=1)
    
    # Mitjana per pacient
    eq_per_pacient = eq_pivot.groupby('SIPCOD')['eq_index_esp'].mean()
    agg_pacient = agg_pacient.join(eq_per_pacient, how='left')
else:
    agg_pacient['eq_index_esp'] = np.nan

# Biomarcadors (última mesura disponible)
bio_vars = ["ALBUMINA", "PCR", "CREATININA"]
for bio_var in bio_vars:
    bio_data = df_pd[df_pd["Variable"] == bio_var].sort_values("Fecha")
    if len(bio_data) > 0:
        bio_last = bio_data.groupby("SIPCOD")["valor_numerico"].last()
        agg_pacient = agg_pacient.join(bio_last.rename(bio_var), how='left')
    else:
        agg_pacient[bio_var] = np.nan

# Netejar i preparar dataset clínic
df_clin = agg_pacient.reset_index()
df_clin_clean = df_clin.dropna(subset=['n_mesures', 'patologia', 'fallecido'])

print(f"Dataset clínic: {len(df_clin_clean)} pacients únics")
print(f"Pacients amb Rockwood: {df_clin_clean['rockwood_mitjana'].notna().sum()}")
print(f"Pacients amb EuroQol: {df_clin_clean['eq_index_esp'].notna().sum()}")
print(f"Taxa de mortalitat: {df_clin_clean['fallecido'].mean()*100:.1f}%")

# VISUALITZACIONS CLÍNIQUES 
# ===================================================================

# 1) Variables clíniques principals
variables_hist = [
    ("rockwood_mitjana", "escala Rockwood mitjana", 15),
    ("eq_index_esp", "índex EuroQol", 20)
]

for var, label, bins in variables_hist:
    data_valid = df_clin_clean.dropna(subset=[var])
    
    if len(data_valid) > 10:
        plt.figure(figsize=(10, 6))
        
        # Histograma 
        color_var = colors_variables.get(var, '#4CAF50')
        plt.hist(data_valid[var], bins=bins, alpha=0.8, color=color_var, 
                edgecolor='black', linewidth=0.5)
        
        plt.xlabel(label, fontsize=12)
        plt.ylabel("Nombre de pacients", fontsize=12)
        plt.title(f"Distribució d\'{label}\n"
                  f"Pacients totals: {len(data_valid):,} | Període: {PERIODE_COMPLET}", 
                  fontweight='bold', fontsize=14)
        plt.grid(alpha=0.3)
        plt.xlim(0,1)
        
        if var == 'eq_index_esp':
            plt.xlim(-0.1, 1.0)  # EuroQol rang optimitzat
        
        # Calcular estadístiques
        mitjana = data_valid[var].mean()
        mediana = data_valid[var].median()
        
        # Afegir estadístiques al gràfic
        textstr = f'Mitjana: {mitjana:.3f}\nMediana: {mediana:.3f}'
        color_caixa = color_var.replace('FF', 'CC') if 'FF' in color_var else color_var
        props = dict(boxstyle='round', facecolor=color_caixa, alpha=0.8)
        plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=10,
                 verticalalignment='top', bbox=props)
        
        plt.tight_layout()
        plt.show()
        
        # Estadístiques descriptives
        print(f"\n{label}:")
        print(f"   n={len(data_valid)}, mitjana={mitjana:.3f} ± {data_valid[var].std():.3f}")
        print(f"   mediana={mediana:.3f}, rang: {data_valid[var].min():.3f} - {data_valid[var].max():.3f}")

# 2) Variables clíniques per tipus de pacient
variables_per_tipus = [
    ("rockwood_mitjana", "escala Rockwood mitjana", 15),
    ("eq_index_esp", "índex EuroQol", 20)
]

for var, label, bins in variables_per_tipus:
    data_valid = df_clin_clean.dropna(subset=[var, 'patologia'])
    
    if len(data_valid) > 10:
        plt.figure(figsize=(12, 6))
        
        # Separar dades per tipus de pacient

        colors_tipus = ['#FF9999', '#66B3FF', '#99FF99'][:len(tipos_a_mantener)]
        
        dades_per_tipus = []
        etiquetes_tipus = []
        
        for tipus in tipos_a_mantener:
            dades_tipus = data_valid[data_valid['patologia'] == tipus][var]
            if len(dades_tipus) > 0:
                dades_per_tipus.append(dades_tipus)
                etiquetes_tipus.append(tipus)
        
        # Histograma amb colors per tipus
        plt.hist(dades_per_tipus, bins=bins, alpha=0.7, label=etiquetes_tipus,
                color=colors_tipus[:len(dades_per_tipus)], edgecolor='black', linewidth=0.5)
        
        plt.xlabel(label, fontsize=12)
        plt.ylabel("Nombre de pacients", fontsize=12)
        plt.title(f"Distribució d\'{label} per tipus de pacient\n"
                  f"Pacients totals: {len(data_valid):,} | Període: {PERIODE_COMPLET}", 
                  fontweight='bold', fontsize=14)
        plt.legend(title="Tipus de pacient", fontsize=10)
        plt.grid(alpha=0.3)
        plt.xlim(0,1)
        
        if var == 'eq_index_esp':
            plt.xlim(-0.1, 1.0)
        
        plt.tight_layout()
        plt.show()
        
        # Estadístiques per tipus
        print(f"\n{label} per tipus de pacient:")
        for i, tipus in enumerate(etiquetes_tipus):
            data_tipus = dades_per_tipus[i]
            print(f"   {tipus}: n={len(data_tipus)}, mitjana={data_tipus.mean():.3f} ± {data_tipus.std():.3f}")

# 3) Biomarcadors
print(f"\nBIOMARCADORS:")

biomarcadors_info = [
    ("ALBUMINA", "Albúmina (g/dL)", 20),
    ("PCR", "PCR (mg/L)", 25),
    ("CREATININA", "Creatinina (mg/dL)", 20)
]

for var, label, bins in biomarcadors_info:
    data_valid = df_clin_clean.dropna(subset=[var])
    
    if len(data_valid) > 10:
        plt.figure(figsize=(10, 6))
        
        # Histograma 
        color_var = colors_variables.get(var, '#FF6B6B')
        plt.hist(data_valid[var], bins=bins, alpha=0.8, color=color_var, 
                edgecolor='black', linewidth=0.5)
        
        plt.xlabel(label, fontsize=12)
        plt.ylabel("Nombre de pacients", fontsize=12)
        plt.title(f"Distribució de {label.lower()}\n"
                  f"Pacients totals: {len(data_valid):,} | Període: {PERIODE_COMPLET}", 
                  fontweight='bold', fontsize=14)
        plt.grid(alpha=0.3)
        
        # Calcular estadístiques
        mitjana = data_valid[var].mean()
        mediana = data_valid[var].median()
        
        # Afegir estadístiques al gràfic 
        textstr = f'Mitjana: {mitjana:.3f}\nMediana: {mediana:.3f}'
        color_caixa = color_var.replace('FF', 'CC') if 'FF' in color_var else color_var
        props = dict(boxstyle='round', facecolor=color_caixa, alpha=0.8)
        plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=10,
                 verticalalignment='top', bbox=props)
        
        plt.tight_layout()
        plt.show()
        
        # Estadístiques descriptives
        print(f"\n{label}:")
        print(f"   n={len(data_valid)}, mitjana={mitjana:.3f} ± {data_valid[var].std():.3f}")
        print(f"   mediana={mediana:.3f}, rang: {data_valid[var].min():.3f} - {data_valid[var].max():.3f}")


# MATRIU DE CORRELACIONS
# ===================================================================

print(f"\n4.1.3 - MATRIU DE CORRELACIONS")
tot_pac= f"3,195"
print("-"*60)

vars_corr = ['n_mesures', 'edat_mitjana', 'eq_index_esp', 'rockwood_mitjana', 
             'ALBUMINA', 'PCR', 'CREATININA']

# Filtrar pacients amb almenys 3 variables disponibles
df_corr = df_clin_clean[vars_corr].dropna(thresh=3)

if len(df_corr) > 20:
    corr_matrix = df_corr.corr()
    
    plt.figure(figsize=(10, 8))
    
    # Màscara per la meitat superior
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    
    sns.heatmap(corr_matrix, 
                mask=mask, 
                annot=True, 
                fmt=".3f", 
                cmap="PRGn", 
                center=0, 
                square=True,
                cbar_kws={"shrink": 0.8},
                linewidths=0.5)
    
    plt.title(f"Matriu de correlacions: variables clíniques\n"
              f"Pacients totals: {tot_pac} | Període: {PERIODE_COMPLET}", 
              fontweight='bold', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Correlacions significatives
    print("\nCorrelacions més rellevants (|r| ≥ 0.3):")
    n_vars = len(vars_corr)
    for i in range(n_vars):
        for j in range(i+1, n_vars):
            var1, var2 = vars_corr[i], vars_corr[j]
            if var1 in corr_matrix.index and var2 in corr_matrix.columns:
                r_value = corr_matrix.loc[var1, var2]
                if abs(r_value) >= 0.3:
                    direccio = "positiva" if r_value > 0 else "negativa"
                    intensitat = "forta" if abs(r_value) >= 0.5 else "moderada"
                    print(f"   {var1} ↔ {var2}: r={r_value:.3f} ({direccio}, {intensitat})")


In [ ]:
# ANÀLISI EUROQOL 
# ===================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats

# CONFIGURACIÓ

sns.set_style("white")

# Configuració del període d'estudi
PERIODE_INICI = "2020"
PERIODE_FI = "2024"
PERIODE_COMPLET = f"{PERIODE_INICI}-{PERIODE_FI}"

colors_variables = {
    'rockwood_mitjana': '#FF9999',      # Rosa 
    'eq_index_esp': '#E6B3FF',          # Lila 
    'ALBUMINA': '#808080',              # Gris
    'PCR': '#808080',                   # Gris
    'CREATININA': '#808080'             # Gris
}

# Colors per EuroQol dimensions
colors_euroqol = [colors_variables['eq_index_esp'], "#FFC000", "#FF6B6B", "#4472C4", "#A569BD"]
colors_nivells = [colors_variables['eq_index_esp'], "#FFC000", "#FF6B6B"]  

# PREPARACIÓ DE DADES EUROQOL
# ===================================================================

# Usar df_pd del código anterior 
if 'df_pd' not in locals():
    if hasattr(df_clean, 'to_pandas'):
        df_pd = df_clean.to_pandas()
    else:
        df_pd = df_clean.copy()
    df_pd["edat_anys"] = df_pd["edad_dias_medicion"] / 365.25

# Variables EuroQol
eq5d_vars = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
             'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']

# Filtrar només EuroQol
df_euroqol = df_pd[df_pd['Variable'].isin(eq5d_vars)].copy()

print(f"Registres EuroQol totals: {len(df_euroqol):,}")
print(f"Pacients amb EuroQol: {df_euroqol['SIPCOD'].nunique():,}")

# Filtrar només mesures completes (5 dimensions per mesura)
mesures_completes = (df_euroqol.groupby(['SIPCOD', 'Fecha'])['Variable']
                     .nunique().reset_index(name='n_dimensions'))
mesures_5d = mesures_completes[mesures_completes['n_dimensions'] == 5]

if len(mesures_5d) > 0:
    mesures_5d['key'] = mesures_5d['SIPCOD'].astype(str) + '_' + mesures_5d['Fecha'].astype(str)
    df_euroqol['key'] = df_euroqol['SIPCOD'].astype(str) + '_' + df_euroqol['Fecha'].astype(str)
    df_euroqol_clean = df_euroqol[df_euroqol['key'].isin(mesures_5d['key'])].copy()
    
    print(f"Mesures completes (5 dimensions): {len(mesures_5d):,}")
    print(f"Registres després de neteja: {len(df_euroqol_clean):,}")
else:
    df_euroqol_clean = df_euroqol.copy()
    print("No s'han trobat mesures completes, usant totes les dades disponibles")

# DISTRIBUCIÓ DE DIMENSIONS EUROQOL
# ===================================================================

# Noms dimensions
noms_dimensions = {
    'EQ5D_Movilidad': 'Mobilitat',
    'EQ5D_CuidadoPersonal': 'Cura Personal', 
    'EQ5D_ActividadesHabituales': 'Activitats',
    'EQ5D_DolorMalestar': 'Dolor',
    'EQ5D_AnsiedadDepresion': 'Ansietat'
}

colors_nivells_lila = ['#F3E5F5', '#CE93D8', '#AB47BC']  # Lila clar -> Lila mitjà -> Lila fosc

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Total de pacients per a les capçaleres
TOTAL_PACIENTS_EQ1 = df_euroqol_clean['SIPCOD'].nunique()
TOTAL_MESURES_EQ = len(mesures_5d)*5

for i, var in enumerate(eq5d_vars):
    var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
    var_name = noms_dimensions[var]
    
    if len(var_data) > 10:
        # Comptar per nivells 
        comptatge_nivells = var_data['valor_ordinal'].value_counts().sort_index()
        percentatges = (comptatge_nivells / len(var_data)) * 100
        
        # Assegurar que tenim els 3 nivells
        for nivell in [1, 2, 3]:
            if nivell not in percentatges.index:
                percentatges[nivell] = 0.0
        
        percentatges = percentatges.sort_index()
        
        # Gràfic 
        ax = axes[i]
        bars = ax.bar(range(len(percentatges)), percentatges.values, 
                     color=colors_nivells_lila[:len(percentatges)],  
                     alpha=0.9, edgecolor='black', linewidth=1)
        
        ax.set_title(f'{var_name}\n', 
                    fontweight='bold', fontsize=12, pad=15)  # Més espai per capçalera
        ax.set_xlabel('Nivell de severitat', fontsize=10)
        ax.set_ylabel('Percentatge (%)', fontsize=10)
        ax.set_xticks(range(len(percentatges)))
        ax.set_xticklabels([f'Nivell {i+1}' for i in range(len(percentatges))])
        ax.grid(axis='y', alpha=0.3)
        
        # Afegir valors a les barres
        for bar, val in zip(bars, percentatges.values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + percentatges.max()*0.02,
                   f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)
        
        # Afegir estadístiques 
        mitjana = var_data['valor_ordinal'].mean()
        textstr = f'Mitjana: {mitjana:.2f}'
        props = dict(boxstyle='round', facecolor='#E1BEE7', alpha=0.8)  # Lila clar
        ax.text(0.85, 0.7, textstr, transform=ax.transAxes, fontsize=9,
                verticalalignment='top', bbox=props)
        
    else:
        axes[i].text(0.5, 0.5, f'{var_name}\nDades insuficients', 
                    ha='center', va='center', transform=axes[i].transAxes, fontsize=12)
        axes[i].set_title(f'{var_name}\nPacients: 0 | Período: {PERIODE_COMPLET}', 
                         fontweight='bold', fontsize=12, pad=15)

# Eliminar subplot extra
if len(eq5d_vars) < len(axes):
    fig.delaxes(axes[-1])

plt.suptitle(f'Distribució de nivells EuroQol-5D per dimensions\n'
             f'Total mesures: {TOTAL_MESURES_EQ:,} | Pacients únics: {TOTAL_PACIENTS_CLINIC_2:,} | Període: {PERIODE_COMPLET}', 
             fontsize=16, fontweight='bold', y=0.98)  # Més alt per evitar sobreposició
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajustar espai per capçalera
plt.show()

# Estadístiques resumides per dimensió 
print(f"\nEstadístiques per dimensió (tota la població):")
for var in eq5d_vars:
    var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
    var_name = noms_dimensions[var]
    if len(var_data) > 10:
        mitjana = var_data['valor_ordinal'].mean()
        mediana = var_data['valor_ordinal'].median()
        print(f"   {var_name}: mitjana={mitjana:.2f}, mediana={mediana:.2f}, n={len(var_data):,}")

noms_dimensions = {
    'EQ5D_Movilidad': 'Mobilitat',
    'EQ5D_CuidadoPersonal': 'Cura Personal', 
    'EQ5D_ActividadesHabituales': 'Activitats',
    'EQ5D_DolorMalestar': 'Dolor',
    'EQ5D_AnsiedadDepresion': 'Ansietat'
}

# TONOS DE LILA PASTEL per als nivells (clar -> fosc)
colors_nivells_lila = ['#F3E5F5', '#CE93D8', '#AB47BC']  # Lila clar -> Lila mitjà -> Lila fosc

# TONOS DE VERD per als nivells del Ministeri (clar -> fosc)
colors_nivells_verd = ['#E8F5E8', '#81C784', '#4CAF50']  # Verd molt clar -> Verd mitjà -> Verd fosc

# DADES DEL MINISTERI DE SANITAT 
# Estructura: {dimensió: [% Nivell 1, % Nivell 2, % Nivell 3]}
dades_ministeri = {
    'EQ5D_Movilidad': [50.14, 37.85, 12.02],        
    'EQ5D_CuidadoPersonal': [74.55, 18.44, 7.01],   
    'EQ5D_ActividadesHabituales': [60.3, 29.55, 10.15],  
    'EQ5D_DolorMalestar': [44.0, 44.99, 10.75],     
    'EQ5D_AnsiedadDepresion': [74.92, 21.24, 3.53]  
}

# GRÀFIC COMPARACIÓ
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

# Total de pacients per a les capçaleres
TOTAL_PACIENTS_EQ1 = df_euroqol_clean['SIPCOD'].nunique()
TOTAL_MESURES_EQ = len(mesures_5d)*5

for i, var in enumerate(eq5d_vars):
    var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
    var_name = noms_dimensions[var]
    
    if len(var_data) > 10:
        # Comptar per nivells
        comptatge_nivells = var_data['valor_ordinal'].value_counts().sort_index()
        percentatges_propis = (comptatge_nivells / len(var_data)) * 100
        
        # Assegurar que tenim els 3 nivells
        for nivell in [1, 2, 3]:
            if nivell not in percentatges_propis.index:
                percentatges_propis[nivell] = 0.0
        
        percentatges_propis = percentatges_propis.sort_index()
        
        # Obtenir dades del ministeri
        percentatges_ministeri = dades_ministeri.get(var, [0, 0, 0])
        
        # Configurar posicions de les barres
        x = np.arange(3)  # Posicions dels nivells 1, 2, 3
        width = 0.35      # Amplada de cada barra
        
        # Gràfic amb COMPARACIÓ
        ax = axes[i]
        
        # Barres morades-UHD 
        bars1 = ax.bar(x - width/2, percentatges_propis.values, width,
                      color=colors_nivells_lila, alpha=0.9, 
                      edgecolor='black', linewidth=1, 
                      label='Dades pròpies')
        
        # Barres verdes-ministeri
        bars2 = ax.bar(x + width/2, percentatges_ministeri, width,
                      color=colors_nivells_verd, alpha=0.9, 
                      edgecolor='black', linewidth=1,
                      label='Ministeri Sanitat')
        
        ax.set_title(f'{var_name}\n', 
                    fontweight='bold', fontsize=12, pad=15)
        ax.set_xlabel('Nivell de severitat', fontsize=10)
        ax.set_ylabel('Percentatge (%)', fontsize=10)
        ax.set_xticks(x)
        ax.set_xticklabels([f'Nivell {i+1}' for i in range(3)])
        ax.grid(axis='y', alpha=0.3)
        
        # Llegenda 
        if i == 0:
            ax.legend(loc='upper left', fontsize=9)
        
        # Afegir valors a les barres morades
        for bar, val in zip(bars1, percentatges_propis.values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{val:.1f}%', ha='center', va='bottom', 
                   fontweight='bold', fontsize=8, color='purple')
        
        # Afegir valors a les barres verdes
        for bar, val in zip(bars2, percentatges_ministeri):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{val:.1f}%', ha='center', va='bottom', 
                   fontweight='bold', fontsize=8, color='darkgreen')
        
        # Afegir estadístiques 
        mitjana = var_data['valor_ordinal'].mean()
        textstr = f'Mitjana: {mitjana:.2f}'
        props = dict(boxstyle='round', facecolor='#E1BEE7', alpha=0.8)
        ax.text(0.85, 0.98, textstr, transform=ax.transAxes, fontsize=9,
                verticalalignment='top', bbox=props)
        
    else:
        axes[i].text(0.5, 0.5, f'{var_name}\nDades insuficients', 
                    ha='center', va='center', transform=axes[i].transAxes, fontsize=12)
        axes[i].set_title(f'{var_name}\n', 
                         fontweight='bold', fontsize=12, pad=15)

# Eliminar subplot extra
if len(eq5d_vars) < len(axes):
    fig.delaxes(axes[-1])

plt.suptitle(f'Comparació EuroQol-5D: Dades pròpies vs Ministeri de Sanitat\n'
             f'Total mesures pròpies: {TOTAL_MESURES_EQ:,} | Pacients únics: {TOTAL_PACIENTS_EQ1:,} | Període: {PERIODE_COMPLET}', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Estadístiques amb comparació
print(f"\nComparació per dimensió:")
print(f"{'Dimensió':<20} {'Pròpia Mitjana':<15} {'Ministeri N1%':<15} {'Ministeri N2%':<15} {'Ministeri N3%':<15}")
print("-" * 80)

for var in eq5d_vars:
    var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
    var_name = noms_dimensions[var]
    if len(var_data) > 10:
        mitjana_propia = var_data['valor_ordinal'].mean()
        dades_min = dades_ministeri.get(var, [0, 0, 0])
        print(f"{var_name:<20} {mitjana_propia:<15.2f} {dades_min[0]:<15.1f} {dades_min[1]:<15.1f} {dades_min[2]:<15.1f}")


import numpy as np
from scipy import stats
from scipy.spatial.distance import jensenshannon
import pandas as pd

# ESTADÍSTICS PER COMPARAR POBLACIONS
# =====================================================================

def calcular_estadistics_comparacio(df_euroqol_clean, dades_ministeri, noms_dimensions, eq5d_vars):
    """
    Calcula diversos estadístics per quantificar diferències entre poblacions
    """
    
    resultats = {}
    

    print("ESTADÍSTICS DE COMPARACIÓ ENTRE POBLACIONS")
    print("=" * 80)
    
    # 1. CHI-QUADRAT TEST 
    print("\n1. TEST CHI-QUADRAT per dimensió:")
    print("-" * 50)
    
    chi2_global = 0
    p_values_chi2 = []
    
    for var in eq5d_vars:
        var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
        var_name = noms_dimensions[var]
        
        if len(var_data) > 10:
            # Observat (mostra pròpia)
            comptatge_observat = var_data['valor_ordinal'].value_counts().sort_index()
            
            # Esperat (població general)
            total_observat = len(var_data)
            dades_min = dades_ministeri[var]
            
            # Assegurar que tenim els 3 nivells
            observat_array = np.zeros(3)
            for i in range(3):
                nivell = i + 1
                if nivell in comptatge_observat.index:
                    observat_array[i] = comptatge_observat[nivell]
            
            # Assegurar que observat suma el total exacte
            total_observat_real = observat_array.sum()
            
            # Calcular esperat proporcional al total observat real
            proporcions_ministeri = np.array(dades_min) / 100
            comptatge_esperat = proporcions_ministeri * total_observat_real
            
            # Ajust per assegurar sumes exactes (arrodoniment)
            comptatge_esperat = np.round(comptatge_esperat).astype(int)
            diferencia = total_observat_real - comptatge_esperat.sum()
            if diferencia != 0:
                # Afegir/treure la diferència al categoria més gran
                idx_max = np.argmax(comptatge_esperat)
                comptatge_esperat[idx_max] += int(diferencia)
            
            # Chi-quadrat test (només si tots els esperats > 5)
            if all(esp >= 5 for esp in comptatge_esperat):
                chi2_stat, p_val = stats.chisquare(observat_array, comptatge_esperat)
            else:
                # Usar test exacte de Fisher (aproximació per chi2 amb Yates)
                chi2_stat, p_val = stats.chisquare(observat_array, comptatge_esperat)
                print(f"      Algun valor esperat < 5 en {var_name}")
            chi2_global += chi2_stat
            p_values_chi2.append(p_val)
            
            # Cramer's V (mida de l'efecte)
            n = total_observat
            cramers_v = np.sqrt(chi2_stat / (n * 2))  # 2 = k-1, on k=3 categories
            
            print(f"{var_name:<15}: χ² = {chi2_stat:.3f}, p = {p_val:.4f}, Cramer's V = {cramers_v:.3f}")
            
            resultats[var] = {
                'chi2': chi2_stat,
                'p_value': p_val,
                'cramers_v': cramers_v
            }
    
    # P-value global ajustat (Bonferroni)
    p_global_bonferroni = min(1.0, min(p_values_chi2) * len(p_values_chi2))
    cramers_v_global = cramers_v.mean()
    print(f"\nχ² GLOBAL: {chi2_global:.3f}")
    print(f"p-value global (Bonferroni): {p_global_bonferroni:.4f}")
    print(f"Cramers V global: {cramers_v_global:.4f}")

    # 2. JENSEN-SHANNON DIVERGENCE 
    print("\n2. JENSEN-SHANNON DIVERGENCE (0 = idèntiques, 1 = totalment diferents):")
    print("-" * 70)
    
    js_divergences = []
    
    for var in eq5d_vars:
        var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
        var_name = noms_dimensions[var]
        
        if len(var_data) > 10:
            # Distribució observada
            comptatge = var_data['valor_ordinal'].value_counts().sort_index()
            dist_observada = np.zeros(3)
            for i in range(3):
                nivell = i + 1
                if nivell in comptatge.index:
                    dist_observada[i] = comptatge[nivell] / len(var_data)
            
            # Distribució esperada (ministeri)
            dist_esperada = np.array(dades_ministeri[var]) / 100
            
            # Jensen-Shannon divergence
            js_div = jensenshannon(dist_observada, dist_esperada)
            js_divergences.append(js_div)
            
            print(f"{var_name:<15}: JS = {js_div:.4f}")
            
            resultats[var]['js_divergence'] = js_div
    
    js_mitjana = np.mean(js_divergences)
    print(f"\nJENSEN-SHANNON MITJANA: {js_mitjana:.4f}")
    
    # 3. DISTÀNCIA HELLINGER
    print("\n3. DISTÀNCIA HELLINGER (0 = idèntiques, √2 ≈ 1.41 = ortogonals):")
    print("-" * 65)
    
    hellinger_distances = []
    
    for var in eq5d_vars:
        var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
        var_name = noms_dimensions[var]
        
        if len(var_data) > 10:
            # Distribució observada
            comptatge = var_data['valor_ordinal'].value_counts().sort_index()
            dist_observada = np.zeros(3)
            for i in range(3):
                nivell = i + 1
                if nivell in comptatge.index:
                    dist_observada[i] = comptatge[nivell] / len(var_data)
            
            # Distribució esperada
            dist_esperada = np.array(dades_ministeri[var]) / 100
            
            # Distància Hellinger
            hellinger = np.sqrt(0.5 * np.sum((np.sqrt(dist_observada) - np.sqrt(dist_esperada))**2))
            hellinger_distances.append(hellinger)
            
            print(f"{var_name:<15}: H = {hellinger:.4f}")
            
            resultats[var]['hellinger'] = hellinger
    
    hellinger_mitjana = np.mean(hellinger_distances)
    print(f"\nHELLINGER MITJANA: {hellinger_mitjana:.4f}")
    
    # 4. ÍNDEX DE DISSIMILITUD 
    print("\n4. ÍNDEX DE DISSIMILITUD (% de casos que caldria redistribuir):")
    print("-" * 65)
    
    dissimilitud_values = []
    
    for var in eq5d_vars:
        var_data = df_euroqol_clean[df_euroqol_clean['Variable'] == var]
        var_name = noms_dimensions[var]
        
        if len(var_data) > 10:
            # Distribució observada
            comptatge = var_data['valor_ordinal'].value_counts().sort_index()
            dist_observada = np.zeros(3)
            for i in range(3):
                nivell = i + 1
                if nivell in comptatge.index:
                    dist_observada[i] = comptatge[nivell] / len(var_data)
            
            # Distribució esperada
            dist_esperada = np.array(dades_ministeri[var]) / 100
            
            # Índex de dissimilitud
            dissimilitud = 0.5 * np.sum(np.abs(dist_observada - dist_esperada))
            dissimilitud_values.append(dissimilitud)
            
            print(f"{var_name:<15}: D = {dissimilitud:.4f} ({dissimilitud*100:.1f}%)")
            
            resultats[var]['dissimilitud'] = dissimilitud
    
    dissimilitud_mitjana = np.mean(dissimilitud_values)
    print(f"\nDISSIMILITUD MITJANA: {dissimilitud_mitjana:.4f} ({dissimilitud_mitjana*100:.1f}%)")
    
    # RESUM 
    # =====================================================================
    print("RESUM ")
    print("=" * 80)
    
    print(f" ESTADÍSTIC PRINCIPAL (Jensen-Shannon): {js_mitjana:.4f}")
    print(f" INTERPRETACIÓ:")
    if js_mitjana < 0.1:
        interpretacio = "Les poblacions són molt similars"
    elif js_mitjana < 0.3:
        interpretacio = "Diferències moderades entre poblacions"
    elif js_mitjana < 0.5:
        interpretacio = "Diferències considerables"
    else:
        interpretacio = "Poblacions molt diferents"
    
    print(f"   {interpretacio}")
    
    print(f"\n DETALLS:")
    print(f"   • Dissimilitud mitjana: {dissimilitud_mitjana*100:.1f}% dels casos caldria redistribuir")
    print(f"   • p-value global: {p_global_bonferroni:.4f} ({'Significatiu' if p_global_bonferroni < 0.05 else 'No significatiu'})")
    
    # Dimensió amb més diferència
    if js_divergences:
        max_js_idx = np.argmax(js_divergences)
        dimensio_mes_diferent = list(noms_dimensions.values())[max_js_idx]
        print(f"   • Dimensió amb més diferència: {dimensio_mes_diferent} (JS = {js_divergences[max_js_idx]:.4f})")
    
    print("\n RECOMANACIÓ:")
    if js_mitjana < 0.2:
        print("   La vostra mostra és representativa de la població general espanyola.")
    else:
        print("   La vostra mostra presenta diferències notables respecte la població general.")
    
    return resultats

# EXECUTAR L'ANÀLISI
resultats_estadistics = calcular_estadistics_comparacio(
    df_euroqol_clean, dades_ministeri, noms_dimensions, eq5d_vars
)

# Crear taula resum
print("TAULA PER DIMENSIÓ")
print("=" * 100)
print(f"{'Dimensió':<15} {'χ²':<8} {'p-val':<8} {'Cramer V':<9} {'Jensen-S':<9} {'Hellinger':<10} {'Dissim %':<9}")
print("-" * 100)

for var in eq5d_vars:
    if var in resultats_estadistics:
        var_name = noms_dimensions[var]
        r = resultats_estadistics[var]
        print(f"{var_name:<15} {r['chi2']:<8.2f} {r['p_value']:<8.4f} {r['cramers_v']:<9.3f} "
              f"{r['js_divergence']:<9.4f} {r['hellinger']:<10.4f} {r['dissimilitud']*100:<9.1f}")

print("\nLLEGENDA:")
print("• χ²: Chi-quadrat (més alt = més diferència)")
print("• p-val: Significació estadística (< 0.05 = significatiu)")
print("• Cramer V: Mida efecte (0-1, >0.3 = efecte gran)")
print("• Jensen-S: Divergència Jensen-Shannon (0-1, >0.3 = molt diferent)")
print("• Hellinger: Distància Hellinger (0-√2, >0.5 = molt diferent)")
print("• Dissim %: % de casos a redistribuir per ser iguals")

#  CÀLCUL I ANÀLISI ÍNDEX EUROQOL 
# ===================================================================
def calcular_euroqol_espanyol(row):
    """Calcula l'índex EuroQol-5D segons els coeficients espanyols"""
    cols = eq5d_vars
    
    if not all(col in row.index and pd.notna(row[col]) for col in cols):
        return np.nan
    
    try:
        valores = [int(row[col]) for col in cols]
        m, c, a, d, an = valores
        
        if all(x == 1 for x in valores):
            return 1.0
        
        # Coeficients espanyols
        decrements = {
            'movilidad': [0, 0.0897, 0.1794],
            'cuidado': [0, 0.1012, 0.2024],
            'actividades': [0, 0.0551, 0.1102], 
            'dolor': [0, 0.0596, 0.1192],
            'ansiedad': [0, 0.0512, 0.1024]
        }
        
        decrement_total = (decrements['movilidad'][min(m-1, 2)] + 
                          decrements['cuidado'][min(c-1, 2)] + 
                          decrements['actividades'][min(a-1, 2)] + 
                          decrements['dolor'][min(d-1, 2)] + 
                          decrements['ansiedad'][min(an-1, 2)])
        
        if any(x == 3 for x in valores):
            decrement_total += 0.2119
        
        index_eq = 1 - 0.1502 - decrement_total
        return max(index_eq, -0.5)
    except:
        return np.nan

# Pivotar dades per calcular índex
if len(df_euroqol_clean) > 0:
    df_eq_pivot = (df_euroqol_clean.pivot_table(
        index=['SIPCOD', 'Fecha', 'sexo', 'fallecido'], 
        columns='Variable', 
        values='valor_ordinal', 
        aggfunc='first'
    ).reset_index())
    
    # Calcular índex si tenim totes les columnes
    if all(col in df_eq_pivot.columns for col in eq5d_vars):
        df_eq_pivot['eq_index_esp'] = df_eq_pivot.apply(calcular_euroqol_espanyol, axis=1)
        df_eq_valid = df_eq_pivot.dropna(subset=['eq_index_esp'])
        
        print(f"Mesures vàlides amb índex calculat: {len(df_eq_valid):,}")
        
        plt.figure(figsize=(12, 6))
        
        plt.hist(df_eq_valid['eq_index_esp'], bins=25, alpha=0.8, 
                color=colors_variables['eq_index_esp'], edgecolor='black', linewidth=1)
        
        plt.xlabel('Índex EuroQol Espanyol', fontsize=12)
        plt.ylabel('Nombre de mesures', fontsize=12)
        plt.title(f'Distribució de l\'índex EuroQol\n'
                  f'Mesures totals: {len(df_eq_valid):,} | Pacients únics: {df_eq_valid["SIPCOD"].nunique():,} | Període: {PERIODE_COMPLET}', 
                  fontweight='bold', fontsize=14)
        plt.xlim(-0.2, 1.05)
        plt.grid(alpha=0.3)
        
        # Afegir estadístiques 
        mitjana = df_eq_valid['eq_index_esp'].mean()
        mediana = df_eq_valid['eq_index_esp'].median()
        textstr = f'Mitjana: {mitjana:.3f}\nMediana: {mediana:.3f}\nDesv. std: {df_eq_valid["eq_index_esp"].std():.3f}'
        color_caixa = colors_variables['eq_index_esp'].replace('FF', 'CC') if 'FF' in colors_variables['eq_index_esp'] else colors_variables['eq_index_esp']
        props = dict(boxstyle='round', facecolor=color_caixa, alpha=0.8)
        plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=11,
                 verticalalignment='top', bbox=props)
        
        plt.tight_layout()
        plt.show()
        
        # Estadístiques descriptives
        print(f"\nEstadístiques índex EuroQol (tota la població):")
        print(f"   n={len(df_eq_valid):,}, mitjana={mitjana:.3f} ± {df_eq_valid['eq_index_esp'].std():.3f}")
        print(f"   mediana={mediana:.3f}, rang: {df_eq_valid['eq_index_esp'].min():.3f} - {df_eq_valid['eq_index_esp'].max():.3f}")
    else:
        print("No es pot calcular l'índex EuroQol - falten dimensions")
        df_eq_valid = pd.DataFrame()

# ANÀLISI PER SEXE I EDAT
# ===================================================================

if len(df_eq_valid) > 20:
    # Boxplot per sexe
    if 'sexo' in df_eq_valid.columns and df_eq_valid['sexo'].notna().sum() > 10:
        plt.figure(figsize=(10, 6))
        
        sns.boxplot(data=df_eq_valid, x='sexo', y='eq_index_esp', color=colors_variables['eq_index_esp'])
        
        plt.xlabel('Sexe', fontsize=12)
        plt.ylabel('Índex EuroQol Espanyol', fontsize=12)
        plt.title(f'Qualitat de vida per sexe\n'
                  f'Mesures totals: {len(df_eq_valid):,} | Pacients únics: {df_eq_valid["SIPCOD"].nunique():,} | Període: {PERIODE_COMPLET}', 
                  fontweight='bold', fontsize=14)
        
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Estadístiques per sexe 
        stats_sexe = df_eq_valid.groupby('sexo')['eq_index_esp'].agg(['count', 'mean', 'std']).round(3)
        print("\nEstadístiques per sexe:")
        for sexe, row in stats_sexe.iterrows():
            print(f"   Sexe {sexe}: n={row['count']}, mitjana={row['mean']:.3f} ± {row['std']:.3f}")
    
    # Scatter plot edat vs EuroQol 
    edat_info = df_pd.groupby('SIPCOD')['edat_anys'].mean().reset_index()
    df_eq_edat = df_eq_valid.merge(edat_info, on='SIPCOD', how='left')
    df_eq_edat = df_eq_edat.dropna(subset=['edat_anys'])
    
    if len(df_eq_edat) > 20:
        plt.figure(figsize=(12, 6))
        
        plt.scatter(df_eq_edat['edat_anys'], df_eq_edat['eq_index_esp'], 
                   c=colors_variables['eq_index_esp'], alpha=0.6, s=50)
        
        # Línia de tendència 
        if len(df_eq_edat) > 5:
            from scipy.stats import linregress
            slope, intercept, r_value, p_value, std_err = linregress(
                df_eq_edat['edat_anys'], df_eq_edat['eq_index_esp'])
            x_line = np.linspace(df_eq_edat['edat_anys'].min(), df_eq_edat['edat_anys'].max(), 100)
            y_line = slope * x_line + intercept
            plt.plot(x_line, y_line, color='red', linestyle='--', alpha=0.8, linewidth=2)
            
            # Afegir correlació al gràfic
            textstr = f'Correlació: r={r_value:.3f}\np-valor: {p_value:.3f}'
            props = dict(boxstyle='round', facecolor='white', alpha=0.8)
            plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=11,
                     verticalalignment='top', bbox=props)
        
        plt.xlabel('Edat (anys)', fontsize=12)
        plt.ylabel('Índex EuroQol Espanyol', fontsize=12)
        plt.title(f'Relació entre edat i qualitat de vida\n'
                  f'Mesures totals: {len(df_eq_edat):,} | Pacients únics: {df_eq_edat["SIPCOD"].nunique():,} | Període: {PERIODE_COMPLET}', 
                  fontweight='bold', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# MATRIU DE CORRELACIONS 
# ===================================================================
if len(df_eq_valid) > 20 and all(col in df_eq_pivot.columns for col in eq5d_vars):
    # Anàlisi de correlacions
    corr_eq = df_eq_pivot[eq5d_vars].corr()
    
    # Canviar noms 
    corr_eq.index = [noms_dimensions[var] for var in corr_eq.index]
    corr_eq.columns = [noms_dimensions[var] for var in corr_eq.columns]
    
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(corr_eq, dtype=bool))
    
    # GRADIENT: 0 = BLANC, 1 = LILA FOSC (correlacions absolutes)
    from matplotlib.colors import LinearSegmentedColormap
    colors_correlacio = ['#FFFFFF', '#F3E5F5', '#E1BEE7', '#CE93D8', '#AB47BC', '#8E24AA']  # Blanc -> Lila fosc
    cmap_correlacio = LinearSegmentedColormap.from_list("blanc_lila_fosc", colors_correlacio)
    
    # Usar valors absoluts per mostrar intensitat de correlació
    corr_abs = corr_eq.abs()
    
    sns.heatmap(corr_abs, mask=mask, annot=corr_eq.values, fmt=".3f",  # Mostrar valors originals
                cmap=cmap_correlacio, vmin=0, vmax=1, square=True,
                cbar_kws={"shrink": 0.8, "label": "Intensitat correlació (valor absolut)"}, 
                linewidths=0.5,
                annot_kws={'fontsize': 10, 'fontweight': 'bold'})
    
    plt.title(f"Matriu de correlacions entre dimensions EuroQol-5D\n"
              f"Mesures totals: {len(df_eq_valid):,} | Pacients únics: {df_eq_valid['SIPCOD'].nunique():,} | Període: {PERIODE_COMPLET}", 
              fontweight='bold', fontsize=14, pad=20)  # Més espai per capçalera
    plt.tight_layout()
    plt.show()
    
    # Correlacions rellevants
    print(f"Correlacions més rellevants (|r| > 0.3):")
    trobades = False
    for i in range(len(corr_eq.index)):
        for j in range(i+1, len(corr_eq.columns)):
            r_value = corr_eq.iloc[i, j]
            if abs(r_value) > 0.3:
                direccio = "positiva" if r_value > 0 else "negativa"
                print(f"   {corr_eq.index[i]} ↔ {corr_eq.columns[j]}: r={r_value:.3f} ({direccio})")
                trobades = True
    if not trobades:
        print("   No s'han trobat correlacions fortes (|r| > 0.3)")

# ===================================================================
# 4.3.6 - GRÀFIC RADIAL
# ===================================================================
if len(df_eq_valid) > 20 and all(col in df_eq_pivot.columns for col in eq5d_vars):
    # Càlcul de mitjanes per tota la població
    mitjanes_dimensions = df_eq_pivot[eq5d_vars].mean()
    
    if len(mitjanes_dimensions) > 0:
        dimensions = [noms_dimensions[var] for var in eq5d_vars]
        angles = np.linspace(0, 2 * np.pi, len(dimensions), endpoint=False).tolist()
        angles += angles[:1]
        
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
        
        ax.set_ylim(1, 3)
        ax.set_yticks([1, 1.5, 2, 2.5, 3])
        ax.set_yticklabels(['1\n(Millor)', '1.5', '2', '2.5', '3\n(Pitjor)'], fontsize=10)
        
        values = mitjanes_dimensions.tolist()
        values += values[:1]
        
        ax.plot(angles, values, 'o-', linewidth=4, label='Tota la població', 
               color=colors_variables['eq_index_esp'], markersize=10, alpha=0.8)
        ax.fill(angles, values, alpha=0.25, color=colors_variables['eq_index_esp'])
        
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(dimensions, fontsize=12, fontweight='bold')
        ax.set_title(f'Perfil radial de dimensions EuroQol\n'
                    f'Mesures totals: {len(df_eq_valid):,} | Pacients únics: {df_eq_valid["SIPCOD"].nunique():,} | Període: {PERIODE_COMPLET}', 
                    fontweight='bold', pad=30, fontsize=16)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# ANÀLISI PER GRUPS D'EDAT
# ===================================================================

print(f"\n4.3.7 - ANÀLISI PER GRUPS D'EDAT")
print("-"*60)

if len(df_eq_edat) > 30:
    # Anàlisi per grups d'edat
    df_eq_edat['grup_edat'] = pd.cut(df_eq_edat['edat_anys'], 
                                    bins=[0, 65, 75, 85, 120], 
                                    labels=['<65 anys', '65-75 anys', '75-85 anys', '>85 anys'])
    
    plt.figure(figsize=(12, 6))

    colors_edat_lila = ['#F3E5F5', '#E1BEE7', '#CE93D8', '#AB47BC']  # Lila molt clar -> Lila fosc
    sns.boxplot(data=df_eq_edat, x='grup_edat', y='eq_index_esp', 
               palette=colors_edat_lila)
    
    plt.xlabel('Grup d\'edat', fontsize=12)
    plt.ylabel('Índex EuroQol', fontsize=12)
    plt.title(f'Qualitat de vida per grups d\'edat\n'
              f'Mesures totals: {len(df_eq_edat):,} | Pacients únics: {df_eq_edat["SIPCOD"].nunique():,} | Període: {PERIODE_COMPLET}', 
              fontweight='bold', fontsize=14, pad=20)
    plt.xticks(rotation=45)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Estadístiques per grup d'edat
    stats_edat = df_eq_edat.groupby('grup_edat')['eq_index_esp'].agg(['count', 'mean', 'std']).round(3)
    print("\nEstadístiques per grup d'edat:")
    for edat, row in stats_edat.iterrows():
        print(f"   {edat}: n={row['count']}, mitjana={row['mean']:.3f} ± {row['std']:.3f}")



In [ ]:
# ANÀLISI TEMPORAL EUROQOL PER TIPUS DE PACIENT
# ===================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

print("APÈNDIX - ANÀLISI TEMPORAL EUROQOL PER TIPUS DE PACIENT")
print("="*60)

colors_variables = {
    'rockwood_mitjana': '#FF9999',      # Rosa 
    'eq_index_esp': '#E6B3FF',          # Lila 
    'ALBUMINA': '#808080',              # Gris
    'PCR': '#808080',                   # Gris
    'CREATININA': '#808080'             # Gris
}

# COLORS ESPECÍFICS PER TIPUS DE PACIENTS 
colors_tipus_pacients = {
    'Crónico pluripatológico': '#FF9999',      # Rosa
    'Paliativo no oncológico': '#66B3FF',      # Blau  
    'Paliativo oncológico': '#99FF99'          # Verd
}

# Funció per obtenir color segons tipus de pacient
def obtenir_color_tipus(tipus_pacient):
    """Retorna el color específic per cada tipus de pacient"""
    for clau, color in colors_tipus_pacients.items():
        if clau.lower() in tipus_pacient.lower():
            return color
    # Colors per defecte si no coincideix amb els tipus coneguts
    return '#E6B3FF'  # Lila 


# NOMBRE DE MESURES EUROQOL PER PACIENT
# ===================================================================

if 'df_eq_valid' in locals() and len(df_eq_valid) > 0:
    print("\n NOMBRE DE MESURES EUROQOL PER PACIENT")
   
    
    # Afegir informació de tipus de pacient
    if 'TIPO_PACIENTE_FUSIONADO' in df_pd.columns:
        tipus_info = df_pd.groupby('SIPCOD')['TIPO_PACIENTE_FUSIONADO'].first().reset_index()
        df_eq_amb_tipus = df_eq_valid.merge(tipus_info, on='SIPCOD', how='left')
    else:
        df_eq_amb_tipus = df_eq_valid.copy()
        df_eq_amb_tipus['TIPO_PACIENTE_FUSIONADO'] = 'Sense informació'
    
    # Contar mesures per pacient
    eq_mesures_pacient = (df_eq_amb_tipus.groupby('SIPCOD')
                         .agg({
                             'eq_index_esp': ['count', 'mean', 'std', 'first', 'last'],
                             'fallecido': 'first',
                             'sexo': 'first',
                             'TIPO_PACIENTE_FUSIONADO': 'first'
                         })
                         .reset_index())
    
    # Aplanar noms
    eq_mesures_pacient.columns = ['SIPCOD', 'eq_n_mesures', 'eq_mitjana', 'eq_std', 
                                  'eq_primera', 'eq_ultima', 'fallecido', 'sexo', 'tipus_pacient']
    
    # Canvi temporal (només per pacients amb múltiples mesures)
    eq_mesures_pacient['canvi_euroqol'] = np.where(
        eq_mesures_pacient['eq_n_mesures'] > 1,
        eq_mesures_pacient['eq_ultima'] - eq_mesures_pacient['eq_primera'],
        np.nan
    )
    
    print(f"Pacients amb EuroQol: {len(eq_mesures_pacient)}")
    print(f"Mitjana de mesures per pacient: {eq_mesures_pacient['eq_n_mesures'].mean():.1f}")
    
    # Gràfic distribució de mesures
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Subplot 1: Distribució general
    mesures_counts = eq_mesures_pacient['eq_n_mesures'].value_counts().sort_index()
    max_mesures = min(12, mesures_counts.index.max())
    
    bars = axes[0].bar(range(1, max_mesures+1), 
                      [mesures_counts.get(i, 0) for i in range(1, max_mesures+1)], 
                      color=colors_variables['eq_index_esp'], alpha=0.8, edgecolor='black')
    
    axes[0].set_xlabel('Nombre de mesures EuroQol', fontweight='bold')
    axes[0].set_ylabel('Nombre de pacients', fontweight='bold')
    axes[0].set_title(f'Distribució del nombre de mesures EuroQol per pacient\n'
                      f'Pacients totals: {len(eq_mesures_pacient):,} | Període: 2020-2024', 
                      fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    axes[0].set_xticks(range(1, max_mesures+1))
    
    # Afegir percentatges
    total = len(eq_mesures_pacient)
    for i, bar in enumerate(bars):
        count = mesures_counts.get(i+1, 0)
        if count > 0:
            pct = count / total * 100
            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(mesures_counts.values)*0.02,
                         f'{count}\n({pct:.1f}%)', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # Subplot 2: Mesures per TIPUS DE PACIENT 
    tipus_pacients = eq_mesures_pacient['tipus_pacient'].value_counts().head(3).index.tolist()
    
    if len(tipus_pacients) >= 2:
        dades_per_tipus = []
        etiquetes_tipus = []
        colors_graf = []  # Per guardar els colors específics
        
        for i, tipus in enumerate(tipus_pacients):
            if i < len(colors_tipus):
                data_tipus = eq_mesures_pacient[eq_mesures_pacient['tipus_pacient'] == tipus]['eq_n_mesures']
                if len(data_tipus) > 0:
                    dades_per_tipus.append(data_tipus)
                    etiquetes_tipus.append(f'{tipus}\n(n={len(data_tipus)})')
        
        if len(dades_per_tipus) >= 2:
           
            axes[1].hist(dades_per_tipus, 
                        bins=np.arange(0.5, max_mesures+1.5, 1),
                        alpha=0.8, color=['#FF9999','#66B3FF','#99FF99'],
                        label=etiquetes_tipus, edgecolor='black', linewidth=0.5)
            
            axes[1].set_xticks(range(1, max_mesures+1))
            axes[1].set_xlabel('Nombre de mesures EuroQol', fontweight='bold')
            axes[1].set_ylabel('Freqüència', fontweight='bold')
            axes[1].set_title(f'Mesures EuroQol per tipus de pacient\n'
                             f'Pacients totals: {sum(len(d) for d in dades_per_tipus):,} | Període: 2020-2024', 
                             fontweight='bold')
            axes[1].legend(title='Tipus de pacient', fontsize=9)
            axes[1].grid(True, alpha=0.3)
        else:
            axes[1].text(0.5, 0.5, 'Dades insuficients\nper comparar tipus',
                        ha='center', va='center', transform=axes[1].transAxes, fontsize=12)
    else:
        axes[1].text(0.5, 0.5, f'Pocs tipus de pacients\ndisponibles\n(n={len(tipus_pacients)})',
                    ha='center', va='center', transform=axes[1].transAxes, fontsize=12)
    
    plt.tight_layout()
    plt.show()

# CANVIS TEMPORALS EN EUROQOL PER TIPUS DE PACIENT
# ===================================================================

if 'eq_mesures_pacient' in locals():
    print("\nA.2 - CANVIS TEMPORALS EN EUROQOL PER TIPUS DE PACIENT")
    print("-"*50)
    
    datos_cambios_eq = eq_mesures_pacient[eq_mesures_pacient['canvi_euroqol'].notna()]
    if len(datos_cambios_eq) > 5:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # Subplot 1: Distribució de canvis 
        axes[0].hist(datos_cambios_eq['canvi_euroqol'], bins=15, alpha=0.8,
                    color=colors_variables['eq_index_esp'], edgecolor='black')
        axes[0].axvline(0, color='black', linestyle='-', linewidth=2, label='Sense canvi')
        axes[0].axvline(datos_cambios_eq['canvi_euroqol'].mean(), color='red', 
                       linestyle='--', linewidth=2, label=f'Mitjana: {datos_cambios_eq["canvi_euroqol"].mean():.3f}')
        
        axes[0].set_xlabel('Canvi en EuroQol (últim - primer)', fontweight='bold')
        axes[0].set_ylabel('Freqüència', fontweight='bold')
        axes[0].set_title(f'Distribució de canvis EuroQol\n'
                          f'Pacients totals: {len(datos_cambios_eq):,} | Període: 2020-2024', 
                          fontweight='bold')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Subplot 2: Canvis per TIPUS DE PACIENT
        tipus_amb_canvis = datos_cambios_eq['tipus_pacient'].value_counts().head(3).index.tolist()
        
        if len(tipus_amb_canvis) >= 2:
            canvis_per_tipus = []
            etiquetes_canvis = []
            colors_canvis = []  # Per guardar els colors específics
            
            for tipus in tipus_amb_canvis:
                canvis_tipus = datos_cambios_eq[datos_cambios_eq['tipus_pacient'] == tipus]['canvi_euroqol']
                if len(canvis_tipus) >= 3:  # Mínim 3 per fer histograma
                    color_tipus = obtenir_color_tipus(tipus)
                    canvis_per_tipus.append(canvis_tipus)
                    etiquetes_canvis.append(f'{tipus}\n(n={len(canvis_tipus)})')
                    colors_canvis.append(color_tipus)
            
            if len(canvis_per_tipus) >= 2:
                axes[1].hist(canvis_per_tipus, bins=8, alpha=0.8,
                            color=colors_canvis,
                            label=etiquetes_canvis, density=True, edgecolor='black', linewidth=0.5)
                axes[1].axvline(0, color='black', linestyle='-', linewidth=2, alpha=0.7)
                for tipus in tipus_amb_canvis:
                    canvis_tipus = datos_cambios_eq[datos_cambios_eq['tipus_pacient'] == tipus]['canvi_euroqol'] 
                    axes[1].axvline(canvis_tipus.mean(), color=colors_tipus_pacients[tipus], 
                    linestyle='--', linewidth=2, alpha=1, label=f'{tipus}: mitjana = {canvis_tipus.mean():.3f}')
                
                axes[1].set_xlabel('Canvi en EuroQol', fontweight='bold')
                axes[1].set_ylabel('Densitat', fontweight='bold')
                axes[1].set_title(f'Canvis EuroQol per tipus de pacient\n'
                                 f'Pacients totals: {sum(len(c) for c in canvis_per_tipus):,} | Període: 2020-2024', 
                                 fontweight='bold')
                axes[1].legend(title='Tipus de pacient', fontsize=9)
                axes[1].grid(True, alpha=0.3)
            else:
                axes[1].text(0.5, 0.5, 'Dades insuficients\nper comparar canvis\nper tipus',
                            ha='center', va='center', transform=axes[1].transAxes, fontsize=12)
        else:
            axes[1].text(0.5, 0.5, 'Pocs tipus amb\nsuficients canvis\ntemporals',
                        ha='center', va='center', transform=axes[1].transAxes, fontsize=12)
        
        plt.tight_layout()
        plt.show()
        
        print(f"Canvis temporals EuroQol:")
        print(f"   Pacients amb múltiples mesures: {len(datos_cambios_eq)}")
        print(f"   Canvi mitjà global: {datos_cambios_eq['canvi_euroqol'].mean():.3f} ± {datos_cambios_eq['canvi_euroqol'].std():.3f}")
        
        # Estadístiques per tipus de pacient
        print(f"\nCanvis per tipus de pacient:")
        for tipus in tipus_amb_canvis:
            canvis_tipus = datos_cambios_eq[datos_cambios_eq['tipus_pacient'] == tipus]['canvi_euroqol']
            if len(canvis_tipus) > 0:
                print(f"   {tipus}: mitjana={canvis_tipus.mean():.3f} ± {canvis_tipus.std():.3f} (n={len(canvis_tipus)})")
    
    else:
        print(f"Nombre insuficient de pacients amb múltiples mesures: {len(datos_cambios_eq)}")


#ESTADÍSTIQUES COMPARATIVES PER TIPUS DE PACIENT
# ===================================================================

if 'eq_mesures_pacient' in locals() and len(eq_mesures_pacient) > 10:
    print("\n ESTADÍSTIQUES COMPARATIVES PER TIPUS DE PACIENT")
 
    
    # Comparar variabilitat entre tipus de pacients
    print("Variabilitat EuroQol per tipus de pacient:")
    
    tipus_principals = eq_mesures_pacient['tipus_pacient'].value_counts().head(5).index.tolist()
    
    for tipus in tipus_principals:
        subset = eq_mesures_pacient[eq_mesures_pacient['tipus_pacient'] == tipus]
        
        if len(subset) >= 5:  # Mínim 5 pacients per tipus
            print(f"\n{tipus}:")
            print(f"   Pacients: {len(subset)}")
            print(f"   Mesures mitjanes per pacient: {subset['eq_n_mesures'].mean():.1f}")
            print(f"   EuroQol mitjà: {subset['eq_mitjana'].mean():.3f} ± {subset['eq_mitjana'].std():.3f}")
            
            # Només per als que tenen múltiples mesures
            subset_multi = subset[subset['eq_n_mesures'] > 1]
            if len(subset_multi) > 0:
                print(f"   Desviació estàndard mitjana: {subset_multi['eq_std'].mean():.3f}")
                print(f"   Pacients amb múltiples mesures: {len(subset_multi)} ({len(subset_multi)/len(subset)*100:.1f}%)")
                
                # Canvis temporals per tipus
                canvis_tipus = subset_multi['canvi_euroqol'].dropna()
                if len(canvis_tipus) > 0:
                    print(f"   Canvi temporal mitjà: {canvis_tipus.mean():.3f} ± {canvis_tipus.std():.3f}")

# RESUM EUROQOL
# ===================================================================

if 'eq_mesures_pacient' in locals() and len(eq_mesures_pacient) > 20:
    print("\nA.4 - GRÀFIC RESUM PER TIPUS DE PACIENT")
    print("-"*50)
    
    # Boxplot EuroQol mitjà per tipus de pacient
    tipus_suficients = eq_mesures_pacient['tipus_pacient'].value_counts()
    tipus_per_grafic = tipus_suficients[tipus_suficients >= 10].index.tolist()[:3]  # Màxim 3 tipus
    
    if len(tipus_per_grafic) >= 2:
        plt.figure(figsize=(12, 6))
        
        data_per_boxplot = eq_mesures_pacient[eq_mesures_pacient['tipus_pacient'].isin(tipus_per_grafic)]
        
        # Paleta de colors específics
        
        sns.boxplot(data=data_per_boxplot, x='tipus_pacient', y='eq_mitjana', 
                   palette=colors_tipus_pacients)
        
        plt.xlabel('Tipus de pacient', fontsize=12, fontweight='bold')
        plt.ylabel('EuroQol mitjà per pacient', fontsize=12, fontweight='bold')
        plt.title(f'Distribució d\'EuroQol per tipus de pacient\n'
                  f'Pacients totals: {len(data_per_boxplot):,} | Període: 2020-2024', 
                  fontweight='bold', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Estadístiques del boxplot
        print("Resum EuroQol per tipus de pacient:")
        for tipus in tipus_per_grafic:
            data_tipus = data_per_boxplot[data_per_boxplot['tipus_pacient'] == tipus]['eq_mitjana']
            print(f"   {tipus}: n={len(data_tipus)}, mitjana={data_tipus.mean():.3f}, mediana={data_tipus.median():.3f}")
    else:
        print(f"Insuficients tipus de pacients amb ≥10 casos per al boxplot")


In [ ]:
# ANÁLISI ROCKWOOD EN VALENCIÀ 
# ===================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats


sns.set_style("white")

# Esquema colors rosa
rosa_principal = "#FF9999"  
rosa_clar = "#FFCCCC"       
rosa_fosc = "#CC6666"        
rosa_suau = "#FFB3B3"      

# Paleta de rosa degradada
colors_rosa = {
    'principal': rosa_principal,
    'clar': rosa_clar, 
    'fosc': rosa_fosc,
    'suau': rosa_suau,
    'gradient1': '#FFE6E6',    
    'gradient2': '#FF6666'   
}

# Crear mapa de colors degradat en rosa
cmap_rosa = LinearSegmentedColormap.from_list('tons_rosa', [rosa_clar, rosa_fosc])

# COLORS ESPECÍFICS PER TIPUS DE PACIENT
colors_tipus_pacient = {
    'Crònic pluripatològic': '#FF9999',      # Rosa
    'Pal·liatiu no oncològic': '#99CCFF',    # Blau
    'Pal·liatiu oncològic': '#99FF99'        # Verd 
}

# PREPARACIÓ DE DADES ROCKWOOD
# ===================================================================

print("\n PREPARACIÓ DE DADES ROCKWOOD")

# Usar df_clean del codi anterior
if 'df_clean' in locals():
    df_pd = df_clean.to_pandas()
    df_pd["edat_anys"] = df_pd["edad_dias_medicion"] / 365.25
else:
    print(" DataFrame df_clean no trobat")

# Filtrar només variables Rockwood
df_rockwood = df_pd[df_pd['Variable'] == 'Rockwood'].copy()

print(f"Registres Rockwood: {len(df_rockwood):,}")
print(f"Pacients amb Rockwood: {df_rockwood['SIPCOD'].nunique():,}")

if len(df_rockwood) == 0:
    print(" No hi ha dades Rockwood disponibles")
else:
    # Crear DataFrame agregat per pacient
    df_rock_pacient = (df_rockwood.groupby('SIPCOD')
                      .agg({
                          'valor_numerico': ['mean', 'count', 'first', 'last', 'std'],
                          'edat_anys': 'mean',
                          'sexo': 'first',
                          'fallecido': 'first',
                          'TIPO_PACIENTE_FUSIONADO': 'first'
                      })
                      .reset_index())
    
    # Aplanar noms de columnes
    df_rock_pacient.columns = ['SIPCOD', 'rockwood_mitjana', 'rockwood_n_mesures', 'rockwood_primera', 
                              'rockwood_ultima', 'rockwood_std', 'edat_anys', 'sexe', 'mort', 'tipus_pacient']
    
    # Variables derivades
    df_rock_pacient['canvi_rockwood'] = np.where(
        df_rock_pacient['rockwood_n_mesures'] > 1,
        df_rock_pacient['rockwood_ultima'] - df_rock_pacient['rockwood_primera'],
        np.nan
    )
    
    # Grups d'edat
    df_rock_pacient['grup_edat'] = pd.cut(
        df_rock_pacient['edat_anys'], 
        bins=[0, 65, 70, 75, 80, 85, 90, 120], 
        labels=['<65', '65-70', '70-75', '75-80', '80-85', '85-90', '>90'],
        right=False
    )
    
    # Filtrar pacients amb dades vàlides
    df_rock_valid = df_rock_pacient[df_rock_pacient['rockwood_mitjana'].notna()].copy()
    
    print(f"Pacients després d'agregació: {len(df_rock_pacient):,}")
    print(f"Pacients amb dades Rockwood vàlides: {len(df_rock_valid):,}")
    
    if len(df_rock_valid) > 0:
        print(f"Estadístiques bàsiques:")
        print(f"   Rockwood mitjà: {df_rock_valid['rockwood_mitjana'].mean():.3f} ± {df_rock_valid['rockwood_mitjana'].std():.3f}")
        print(f"   Mediana: {df_rock_valid['rockwood_mitjana'].median():.3f}")
        print(f"   Rang: {df_rock_valid['rockwood_mitjana'].min():.3f} - {df_rock_valid['rockwood_mitjana'].max():.3f}")

# ROCKWOOD PER GRUPS D'EDAT
# ===================================================================

print(f"\n ROCKWOOD PER GRUPS D'EDAT")
print("-"*60)

if 'df_rock_valid' in locals() and len(df_rock_valid) > 20:
    plt.figure(figsize=(12, 6))
    
    # Boxplot per edat 
    sns.boxplot(data=df_rock_valid, x='grup_edat', y='rockwood_mitjana',
               color=rosa_principal, linewidth=2)
    
    plt.xlabel('Grup d\'edat', fontsize=12, fontweight='bold')
    plt.ylabel('Escala Rockwood (0-1)', fontsize=12, fontweight='bold')
    plt.title('Distribució de Rockwood per grups d\'edat \n Pacients totals:' f'{df_rockwood['SIPCOD'].nunique():,} | Període: {PERIODE_COMPLET}', fontsize=14, fontweight='bold')
    plt.ylim(0,1)

    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # Estadístiques per grup d'edat
    print(f"Rockwood mitjà per grup d'edat:")
    for grup in sorted(df_rock_valid['grup_edat'].dropna().unique()):
        data_grup = df_rock_valid[df_rock_valid['grup_edat'] == grup]
        if len(data_grup) > 0:
            print(f"    {grup}: n={len(data_grup)}, mitjana={data_grup['rockwood_mitjana'].mean():.3f}")

